In [2]:
!pip install streamlit transformers torch torchvision matplotlib pillow





In [3]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from IPython.display import display, clear_output
import ipywidgets as widgets

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda" if torch.cuda.is_available() else "cpu")

# Set your image folder path here
base_folder = "C:\\Users\\RUHIN CHHIPA\\OneDrive\\Desktop\\main_imgFolder"  # inside this, you can have folders like nature/, objects/, etc.

# List all subfolders
all_folders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]
selected_folder = widgets.Dropdown(options=all_folders, description='Folder:')

# Placeholder for image paths
all_images_path = []
index = 0
out = widgets.Output()

# Generate caption using BLIP
def generate_caption(image_path):
    raw_image = Image.open(image_path).convert('RGB')
    inputs = processor(raw_image, return_tensors="pt").to(model.device)
    out = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_p=0.9)
    return processor.decode(out[0], skip_special_tokens=True)

# Show image with caption
def show_image_with_caption():
    with out:
        clear_output(wait=True)
        if all_images_path:
            image_path = all_images_path[index]
            img = Image.open(image_path)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"Caption: {generate_caption(image_path)}")
            plt.show()

# Update image list on folder change
def on_folder_change(change):
    global all_images_path, index
    folder = change['new']
    folder_path = os.path.join(base_folder, folder)
    all_images_path = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    index = 0
    show_image_with_caption()

# Navigation buttons
def on_next_clicked(b):
    global index
    if index < len(all_images_path) - 1:
        index += 1
        show_image_with_caption()

def on_prev_clicked(b):
    global index
    if index > 0:
        index -= 1
        show_image_with_caption()

# Button setup
next_button = widgets.Button(description='Next ▶️', button_style='success')
prev_button = widgets.Button(description='◀️ Previous', button_style='info')
next_button.on_click(on_next_clicked)
prev_button.on_click(on_prev_clicked)

selected_folder.observe(on_folder_change, names='value')

# Display interface
display(widgets.HBox([prev_button, next_button]))
display(selected_folder)
display(out)

# Initial call to load folder
on_folder_change({'new': selected_folder.value})


Dropdown(description='Folder:', options=('daily_life', 'nature', 'object'), value='daily_life')

Output()